# Convolutional Neural Network (CNN)

##### In this code it is implemented an artificial neural network for a multiclass classification task.
##### In particular it is used pytorch and to extract the features from the images are implemented convolutional layers


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#extract
!tar -xf /content/drive/MyDrive/Supervised_Learning/Exam/annot.tar
!tar -xf /content/drive/MyDrive/Supervised_Learning/Exam/train.tar
!tar -xf /content/drive/MyDrive/Supervised_Learning/Exam/val.tar

In [ ]:
# Select the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Select the batch size
bs = 32

In [ ]:
# Initialize the paths of the folders in which the images are stored
train_dir = '/content/train_set'
test_dir = '/content/val_set'

In [ ]:
class FoodDataset(Dataset):
  def __init__(self, root, df, split=None, transform=None):
    self.root = root
    self.split = split
    self.transform = transform
    self.df = df

    # Compute the length of the dataset
    total_data_len = len(df)

    # Randomly shuffle indices for train/val/test split
    idx = np.arange(total_data_len, dtype=int)
    np.random.seed(0)
    np.random.shuffle(idx)

    # Perform the train/val split (80/20)
    if self.split == "train":
        self.im_dir = [os.path.join(self.root, self.df.iloc[i, 0]) for i in idx[:int(total_data_len * 0.8)]]
        self.target = [self.df.iloc[i, 1] for i in idx[:int(total_data_len * 0.8)]]
    elif self.split == "val":
        self.im_dir = [os.path.join(self.root, self.df.iloc[i, 0]) for i in idx[int(total_data_len * 0.8):]]
        self.target = [self.df.iloc[i, 1] for i in idx[int(total_data_len * 0.8):]]
    # Take all the images in the test folder
    elif self.split == "test":
        self.im_dir = [os.path.join(self.root, self.df.iloc[i, 0]) for i in idx[:]]
        self.target = [self.df.iloc[i, 1] for i in idx[:]]

  def __getitem__(self, idx):

    image = Image.open(self.im_dir[idx])
    label = self.target[idx]

    if self.transform:
      image = self.transform(image)

    return image, label

  def __len__(self):
    return len(self.im_dir)

In [ ]:
# Initialize the dataset
df=pd.read_csv('/content/train_info.csv', header=None)

In [ ]:
file = '/content/class_list.txt'

# open the file in read mode
with open(file, 'r') as file:
    # read lines from the file
    lines = file.readlines()

# initialize an empty dictionary
classes = {}

# iterate through each line and split key-value pairs
for line in lines:
    key, value = line.strip().split(' ')
    classes[key.strip()] = value.strip()

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally for training augmentation
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation((0,180)),  # Rotate images randomly by 0 to 180 degrees
    transforms.RandomPerspective(),  # Apply random perspective transformation
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), ),  # Apply random affine transformation
    transforms.ToTensor(),  # Convert PIL images to PyTorch tensors
    transforms.Normalize(  # Normalize pixel values based on ImageNet statistics
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

transform_val = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256 (consistent with training)
    transforms.ToTensor(),  # Convert PIL images to PyTorch tensors
    transforms.Normalize(  # Normalize pixel values using the same statistics
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
# Initialize the train/val dataset
train_ds_transformed = FoodDataset(train_dir, df, 'train', transform=transform_train)
val_ds_transformed = FoodDataset(train_dir, df, 'val', transform=transform_val)

In [ ]:
# Select one image to check if the dataset class works properly
image, label = train_ds_transformed[103]
# Print the label and the image (if the image is a tensor, then it worked)
print(classes[str(label)])
image

beignet


tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1

In [ ]:
# Initialize the train/val data loader
train_loader = DataLoader(train_ds_transformed, batch_size=bs, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds_transformed, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() # Call the superclass constructor
        self.conv1 = nn.Conv2d(3, 20, 9, stride=3) # First convolutional layer
        self.pool = nn.MaxPool2d(2, 2) # Max pooling layer
        self.conv2 = nn.Conv2d(20, 40, 9, stride=2) # Second convolutional layer
        self.conv3 = nn.Conv2d(40, 80, 3, stride=1, padding=1) # Third convolutional layer
        self.fc1 = nn.Linear(80*4*4, 500)  # First fully-connected layer
        self.dropout = nn.Dropout(0.25) # Dropout layer
        self.fc2 = nn.Linear(500, 346) # Second fully-connected layer
        self.fc3 = nn.Linear(346, 251) # Third fully-connected layer

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = self.pool(F.leaky_relu(self.conv3(x)))
        # print(x.shape)
        x = x.view(x.shape[0],-1) # Flatten the output from convolutional layers
        # print(x.shape)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x) # Output layer
        return F.log_softmax(x, dim=1) # Apply log softmax to the output

In [ ]:
# Load the network for training or inference (or both)
if os.path.exists('/content/FoodNet.pth'):
    net = Net()
    net.load_state_dict(torch.load('/content/FoodNet.pth', map_location=device))
    net.to(device)
    print("Succesfully loaded FoodNet")
else:
    net = Net()
    net.to(device)
    print("FoodNet initialized correctly")

Succesfully loaded FoodNet


In [ ]:
from torchsummary import summary
# Print model summary
summary(net, (3, 256, 256))  # Input shape (channels, height, width)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 83, 83]           4,880
         MaxPool2d-2           [-1, 20, 41, 41]               0
            Conv2d-3           [-1, 40, 17, 17]          64,840
         MaxPool2d-4             [-1, 40, 8, 8]               0
            Conv2d-5             [-1, 80, 8, 8]          28,880
         MaxPool2d-6             [-1, 80, 4, 4]               0
           Dropout-7                 [-1, 1280]               0
            Linear-8                  [-1, 500]         640,500
           Dropout-9                  [-1, 500]               0
           Linear-10                  [-1, 346]         173,346
           Linear-11                  [-1, 251]          87,097
Total params: 999,543
Trainable params: 999,543
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/

In [ ]:
import torch.optim as optim  # Optimization algorithms for training the model
import torch.nn.functional as F  # Common loss functions and activation functions
import itertools  # Utility functions for generating combinations
from torch.optim.lr_scheduler import CosineAnnealingLR  # Learning rate scheduler for training
import matplotlib.pyplot as plt  # Plotting library for visualization


# Define training parameters (epochs, loss function, optimizer, and scheduler)
epochs = 100  # Number of training epochs
criterion = nn.CrossEntropyLoss()  # Crossentropyloss function for classification
optimizer = optim.Adam(net.parameters(), lr=0.0001)  # Adam optimizer with learning rate 0.001
scheduler = CosineAnnealingLR(optimizer,
                              T_max=len(train_loader) * epochs,  # Maximum number of iterations for scheduler
                              eta_min=1e-8)  # Minimum learning rate for scheduler

In [ ]:
best_val_loss = float('inf') # Initialize the validation loss
patience = 10 # Initialize the patience level
patience_counter = 0 # Initialize the counter

for epoch in range(epochs):
    # Initialize Running loss, ground truth labels and predictions
    running_loss = []
    gt = []
    pred = []

    # Set the network in training mode
    net.train()

    for i, inp in enumerate(train_loader):
        inputs, labels = inp
        gt.append(labels.cpu().numpy())   # Append ground truth

        inputs = inputs.to(device)  # Move data to the appropriate device (CPU or GPU)
        labels = labels.to(device)  # Move labels to the appropriate device

        # Zero the parameter gradients before each backward pass
        optimizer.zero_grad()

        # Forward pass, calculate loss
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pred.append(torch.max(outputs.cpu(), 1)[1])

        # Backward pass and parameter update
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model weights based on gradients
        scheduler.step()  # Update learning rate according to the scheduler

        # Print statistics (every 10% of the training data)
        running_loss.append(loss.item())
        if (i + 1) % (len(train_loader) // 10) == 0:  # Every 10% of the epoch
            gt = np.stack(list(itertools.chain.from_iterable(gt))).squeeze()  # Combine ground truth labels
            pred = np.stack(list(itertools.chain.from_iterable(pred))).squeeze()  # Combine predictions
            acc = np.sum(gt==pred)/len(gt)
            print('%d, [%d, %d] loss: %.4f\tAccuracy: %.3f\tlr: %.6f' %
                      (epoch + 1, i + 1, len(train_loader), np.mean(running_loss), acc, optimizer.param_groups[-1]['lr']))

            running_loss = []
            gt = []
            pred = []

    # Validation loop (after each training epoch)
    running_loss = []  # List to store validation loss for each batch
    gt = []  # List to store ground truth labels (predicted age)
    pred = []  # List to store predicted labels (model output)

    net.eval()  # Set the model to evaluation mode (deactivates dropout and other training behaviors)

    for i, data in enumerate(val_loader):
        # Get inputs and labels from the data loader
        inputs, labels = data
        gt.append(labels.item())  # Append ground truth as single values

        inputs = inputs.to(device)  # Move data to the appropriate device
        labels = labels.to(device)  # Move labels to the appropriate device

        # Forward pass with gradient suppression
        with torch.no_grad():
            outputs = net(inputs)  # Get model predictions without calculating gradients

        pred.append(torch.max(outputs.cpu(), 1)[1])
        loss = criterion(outputs, labels)
        running_loss.append(loss.item())

    # Calculate and print validation performance metrics
    gt = np.stack(list(gt)).squeeze()
    pred = np.stack(list(pred)).squeeze()
    acc = np.sum(gt==pred)/len(gt)

    print(gt, pred)
    print(np.sum(gt==pred),len(gt))

    # Compute validation loss
    val_loss = np.mean(running_loss)
    print('Validation loss: %.6f\tAccuracy: %.3f' % (val_loss, acc))

    # Early stop of the train in case of 10 consecutive non-increasing in validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(net.state_dict(), "FoodNet.pth") # Save the best model
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch} due to no improvement in validation loss for {patience} epochs.")
            break

print('Finished Training')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


KeyboardInterrupt: 

In [ ]:
# Initialize the test dataframe
test_df = pd.read_csv('/content/val_info.csv')

In [ ]:
# Initialize the test dataset
test_ds_transformed = FoodDataset(test_dir, test_df, 'test', transform=transform_val)

In [ ]:
# Initialize the test data loader
test_loader = DataLoader(test_ds_transformed, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
# Check if the image is correct
image, label = test_ds_transformed[103]

print(classes[str(label)])
image

pad_thai


tensor([[[ 2.0777,  2.0777,  2.1462,  ...,  2.2147,  2.1975,  2.1804],
         [ 2.0948,  2.1119,  2.1462,  ...,  2.2489,  2.2489,  2.2318],
         [ 2.1975,  2.1975,  2.2147,  ...,  2.2489,  2.2489,  2.2489],
         ...,
         [ 0.6221,  0.5707,  0.5193,  ...,  0.5364,  0.6392,  0.5364],
         [ 0.6563,  0.6221,  0.5878,  ...,  0.5193,  0.6563,  0.5022],
         [ 0.7591,  0.7248,  0.6563,  ...,  0.4679,  0.6221,  0.4337]],

        [[ 1.6408,  1.6408,  1.6408,  ...,  1.7283,  1.7283,  1.7108],
         [ 1.6057,  1.6057,  1.5882,  ...,  1.7808,  1.7808,  1.7633],
         [ 1.6232,  1.6232,  1.6057,  ...,  1.8333,  1.8158,  1.8158],
         ...,
         [ 0.0126, -0.0049, -0.0749,  ...,  0.2227,  0.3627,  0.2927],
         [ 0.0826,  0.0476, -0.0049,  ...,  0.2052,  0.3803,  0.2577],
         [ 0.1877,  0.1527,  0.1176,  ...,  0.1877,  0.3452,  0.1877]],

        [[ 1.1934,  1.1934,  1.1759,  ...,  1.2631,  1.2108,  1.1934],
         [ 1.1411,  1.1411,  1.1062,  ...,  1

In [ ]:
# Testing loop
running_loss = []  # List to store test loss for each batch
gt_labels = []  # List to store ground truth labels
pr_labels = []  # List to store predicted labels
top5_correct = 0  # Counter for top-5 accuracy

net.eval()  # Set the model to evaluation mode

for i, data in enumerate(test_loader):
    # Get inputs and labels from the data loader
    inputs, labels = data
    gt_labels.append(labels.item())  # Append ground truth as single values

    inputs = inputs.to(device)  # Move data to the appropriate device
    labels = labels.to(device)  # Move labels to the appropriate device

    # Forward pass with gradient suppression
    with torch.no_grad():
        outputs = net(inputs)  # Get model predictions without calculating gradients

    pr_labels.append(torch.max(outputs.cpu(), 1)[1])
    loss = criterion(outputs, labels)
    running_loss.append(loss.item())

    # Calculate top-5 accuracy
    top5_preds = torch.topk(outputs, 5, dim=1).indices.cpu().numpy()
    if labels.cpu().numpy() in top5_preds:
        top5_correct += 1

# Calculate and print validation performance metrics
gt = np.stack(list(gt_labels)).squeeze()
pred = np.stack(list(pr_labels)).squeeze()
test_loss = np.mean(running_loss)

from sklearn.metrics import accuracy_score, precision_score, recall_score
acc = accuracy_score(gt, pred)
prec = precision_score(gt, pred, average='macro')
rec = recall_score(gt, pred, average='macro')
top5_acc = top5_correct / len(test_loader.dataset)

# Display the outcomes
print(f'Test loss: {test_loss:.6f}\t Accuracy: {acc:.3f}\tPrecision: {prec:.3f}\tRecall: {rec:.3f}\tTop-5 Accuracy: {top5_acc:.3f}')


[180 176  64 ...  58 209   6] [168 118 172 ...  34  69 218]
2769 11993
Test loss: 3.461395	Accuracy: 0.231
Test loss: 3.461395	 Accuracy: 0.231	Precision: 0.221	Recall: 0.222	Top-5 Accuracy: 0.490
